In [90]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import peak_signal_noise_ratio as PSNR
from imutils.paths import list_images
import imutils
import cv2
import tensorlayer as tl
import os
from model import get_G, get_D

In [91]:
def load_dataset(path):
    img_list = list(list_images(path))
    imgs = tl.vis.read_images(img_list,printable=False,n_threads=32)
    return imgs

In [131]:
def compute_score(path_ori, path_sr):
    ori_imgs = load_dataset(path_ori)
    sr_imgs = load_dataset(path_sr)
    PSNR_scores = []
    SSIM_scores = []
    for ori_img,sr_img in zip(ori_imgs,sr_imgs):
        print(ori_img.shape) # height width
        print(sr_img.shape)
        height, width = ori_img.shape[:2]
        if height%4 != 0:
            height = height-height%4
        if width%4 != 0:
            width = width-width%4
        ori_img=ori_img[:height,:width]
        PSNR_scores.append(PSNR(ori_img,sr_img))
        SSIM_scores.append(SSIM(ori_img,sr_img, multichannel=True))
    mean_PSNR_score = np.mean(np.array(PSNR_scores))
    mean_SSIM_score = np.mean(np.array(SSIM_scores))
    return mean_PSNR_score, mean_SSIM_score

In [101]:
def gen_sr():    
    set5_lr_img_list = list(list_images('archive/Set5_lr'))
    set14_lr_img_list = list(list_images('archive/Set14_lr'))
    set5_lr_imgs = tl.vis.read_images(set5_lr_img_list, n_threads=32)
    set14_lr_imgs = tl.vis.read_images(set14_lr_img_list, n_threads=32)

    name5 = [os.path.split(path)[-1] for path in set5_lr_img_list]
    name14 = [os.path.split(path)[-1] for path in set14_lr_img_list]

    ###========================== DEFINE MODEL ============================###
    temp5 = []
    temp14 = []
    for img in set5_lr_imgs:
        normal_img = img/127.5 - 1.
        temp5.append(normal_img)
    for img in set14_lr_imgs:
        normal_img = img/127.5 - 1.
        temp14.append(normal_img)

    G = get_G([None, None, 3])
    G.load_weights('models/g_20.h5')

    for img,name in zip(temp5,name5):
        img = img[np.newaxis,:,:,:]
        out = G(img).numpy()
        tl.vis.save_image(out[0], os.path.join('archive/Set5_sr', name))
    for img,name in zip(temp14,name14):
        img = img[np.newaxis,:,:,:]
        out = G(img).numpy()
        tl.vis.save_image(out[0], os.path.join('archive/Set14_sr', name))

In [99]:
# path_ori = 'archive/Set5'
# path_lr = 'archive/Set5_lr'
# path_bicubic = 'archive/Set5_bicubic'
# img_list = list(list_images(path_ori))
# for path in img_list:
#     img = cv2.imread(path)
#     width = img.shape[1]
#     name = os.path.split(path)[-1]
    
#     img_lr = imutils.resize(img,width=width//4)
#     img_bicubic = imutils.resize(img_lr,width=width)

#     new_path_lr = path_lr + '/' + name
#     new_path_bicubic = path_bicubic + '/' + name
#     cv2.imwrite(new_path_lr, img_lr)
#     cv2.imwrite(new_path_bicubic, img_bicubic)    

In [128]:
path_ori = 'archive/Set14'
path_lr = 'archive/Set14_lr'
path_bicubic = 'archive/Set14_bicubic'
img_list = list(list_images(path_ori))
for path in img_list:
    img = cv2.imread(path)
    height, width = img.shape[:2]
    if height%4 != 0:
        height = height-height%4
    if width%4 != 0:
        width = width-width%4
    img=img[:height,:width]
    width = img.shape[1]
    name = os.path.split(path)[-1]
    
    img_lr = imutils.resize(img,width=width//4)
    img_bicubic = imutils.resize(img_lr,width=width)

    new_path_lr = path_lr + '/' + name
    new_path_bicubic = path_bicubic + '/' + name
    cv2.imwrite(new_path_lr, img_lr)
    cv2.imwrite(new_path_bicubic, img_bicubic)    

In [126]:
x = np.ones((301,401,3))

In [ ]:
height,width = x.shape[:2]

In [127]:
x = x[:300,:]
x.shape

(300, 401)

In [121]:
if __name__ == '__main__':
    import argparse
    ap = argparse.ArgumentParser()
    ap.add_argument("-hr", "--high-reso", type=str, default='archive/Set5', help="path to high resolution images directory")
    ap.add_argument("-sr", "--super-reso", type=str, default='archive/Set5_sr', help="path to super resolution images directory")

    args = vars(ap.parse_args())

    PSNR_score, SSIM_score = compute_score(args['high_reso'], args['super_reso'])
    print('PSNR_score = ',PSNR_score)
    print('SSIM_score = ',SSIM_score)    

usage: ipykernel_launcher.py [-h] [-hr HIGH_RESO] [-sr SUPER_RESO]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\quangmx\AppData\Roaming\jupyter\runtime\kernel-b4aac030-14e8-486a-88cd-646af7e88a83.json


SystemExit: 2

In [129]:
gen_sr()

[TL] read 5 from 
[TL] read 14 from 

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Lossy conversion from float32 to uint8. Range [-0.9677249789237976, 0.9679699540138245]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9692542552947998, 0.9245551824569702]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.8928486704826355, 0.843512237071991]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9645062685012817, 0.9503329992294312]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9612782597541809, 0.9301753044128418]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9459552764892578, 0.937342643737793]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9412535429000854, 0.92133176

In [133]:
PSNR_score, SSIM_score = compute_score('archive/Set5', 'archive/Set5_sr')
print('Set5 PSNR_score = ',PSNR_score)
print('Set5 SSIM_score = ',SSIM_score)

PSNR_score, SSIM_score = compute_score('archive/Set14', 'archive/Set14_sr')
print('Set14 PSNR_score = ',PSNR_score)
print('Set14 SSIM_score = ',SSIM_score)   

(512, 512, 3)
(512, 512, 3)
(288, 288, 3)
(288, 288, 3)
(256, 256, 3)
(256, 256, 3)
(280, 280, 3)
(280, 280, 3)
(344, 228, 3)
(344, 228, 3)
Set5 PSNR_score =  26.423144211802004
Set5 SSIM_score =  0.7920979810592456
(480, 500, 3)
(480, 500, 3)
(576, 720, 3)
(576, 720, 3)
(512, 512, 3)
(512, 512, 3)
(288, 352, 3)
(288, 352, 3)
(361, 250, 3)
(360, 248, 3)
(276, 276, 3)
(276, 276, 3)
(362, 500, 3)
(360, 500, 3)
(288, 352, 3)
(288, 352, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 768, 3)
(512, 768, 3)
(512, 512, 3)
(512, 512, 3)
(656, 529, 3)
(656, 528, 3)
(391, 586, 3)
(388, 584, 3)
Set14 PSNR_score =  23.691334195431107
Set14 SSIM_score =  0.688402740312762


In [ ]:
with open('g_losses.txt') as f:
    lines = [line.rstrip() for line in f]
lines=np.asarray(lines,dtype=np.float)
i = np.asarray(list(range(1,lines.shape[0]+1)))

import matplotlib.pyplot as plt
plt.plot(i,lines)